In [10]:
import astropy
from astropy.io import fits 
from astropy.stats import sigma_clip
from astropy.table import Table
from astropy.time import Time
from astropy.visualization import time_support
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as sig
from scipy.stats import sigmaclip

In [11]:
def dataClean(filename): 
    """ This function will remove nan values from TESS 
        20 second lightcurve data and correct time values
        Stage: complete"""
    
    #Create an array of time and flux data with nans removed    
    with fits.open(filename, mode="readonly") as hdulist:
        raw_time = hdulist[1].data['TIME']
        raw_flux = hdulist[1].data['PDCSAP_FLUX']
        raw_err = hdulist[1].data['PDCSAP_FLUX_ERR']
    data = np.vstack((raw_time, raw_flux, raw_err))
    nonan_data = data[:, ~np.isnan(data).any(axis=0)]

    #Apply time correction 
    times = nonan_data[0]
    flux = nonan_data[1]
    error = nonan_data[2]
    t_corr = []
    for i in times:
        r = i + 2457000
        t_corr.append(r)
    time = Time(t_corr, format = 'jd', scale = 'utc')
    time.format = 'iso'
    
    #Create arrays of cleaned data
    time = np.array(time)
    flux = np.array(flux)
    err = np.array(error)
    
    #Return cleaned data
    return[time,flux,err]

In [12]:
def generate_Qcurve(cleaned_data):
    """ This function will take cleaned data and dataframes of quiescent light curve
    as well as flaring light curve values
    Stage: complete"""

    #Identify quiescent light curve
    flux_smooth = sig.savgol_filter(cleaned_data[1], 1400, 3) #my opinion of the best parameters for this data
    index = np.where(cleaned_data[1] < (flux_smooth + (3*cleaned_data[2]))) #using 3 as the significance for the error for now
    q_time = cleaned_data[0][index] # time associated w/ quiescent flux
    q_flux = sig.savgol_filter(cleaned_data[1][index],2000,3) # quiescent flux
    ind2 = np.where(q_flux) #index created to assure lengths of data sets are equal
    
    #Set and index variables
    fluxes = cleaned_data[1]
    times = cleaned_data[0]
    F_err = cleaned_data[2] 
    flux_list = fluxes[ind2]
    time_list = times[ind2]
    err_list = F_err[ind2]

    #Ensure lists are same size for flare and quiescent data & have same timestamps at same indices
    qIndex = np.where(q_time) 
    err_list = err_list[qIndex]
    q_time = times[qIndex]
    q_flux = sig.savgol_filter(flux_list[qIndex],2000,3)
    
    #Create dataframes of Quiescent and Flaring lightcurves
    quiescence = pd.DataFrame({
                            'Time': q_time,
                            'Quiescent Flux':q_flux
    })

    cd = pd.DataFrame({
                                'Time': time_list[qIndex],
                                'Flux':flux_list[qIndex],
                                'Flux Error': err_list[qIndex]
    })
    return[quiescence,cd]

In [13]:
def find_ix_ranges(ix, buffer=False):
    """ Finds indexes in the range.
    
        From MC GALEX function defs"""
    
    foo, bar = [], []
    for n, i in enumerate(ix):
        if len(bar) == 0 or bar[-1] == i-1:
            bar += [i]
        else:
            if buffer:
                bar.append(min(bar)-1)
                bar.append(max(bar)+1)
            foo += [np.sort(bar).tolist()]
            bar = [i]
        if n == len(ix)-1:
            if buffer:
                bar.append(min(bar)-1)
                bar.append(max(bar)+1)
            foo += [np.sort(bar).tolist()]
    return foo

In [ ]:
def get_inff(lc, clipsigma=3, quiet=True, band='NUV',
             binsize=30.):
    """ Calculates the Instantaneous Non-Flare Flux values.
    
        From MC GALEX function defs"""
    
    sclip = sigma_clip(np.array(lc['Flux']), sigma=clipsigma)
    inff = np.ma.median(sclip)
    inff_err = np.sqrt(inff*len(sclip)*binsize)/(len(sclip)*binsize)
    if inff and not quiet:
        print('Quiescent at {m} AB mag.'.format(m=gt.counts2mag(inff, band)))
    return inff, inff_err

In [75]:
def flareFinder(curve,q_curve,sig):
    """This function will run through the data to find 
    flares and ranges of flares. This function will return
    a table of flares ranges. 
    
    Adapted from MC GALEX function defs.
    Stage: debugging"""
    
    fluxes = curve['Flux']
    for flux in fluxes:
        ix = np.where(((np.array(curve['Flux'].values)-(sig*np.array(curve['Flux Error'].values)) >= q_curve['Quiescent Flux'])))[0]
        flareFlux = ix
        flux_ix = []
        
        for ix_range in find_ix_ranges(ix):
            # go backwards
            consec = 0 
            err = curve.iloc[ix_range[0]]['Flux Error'] 
            
            #while flux - err > quiescence, find 2 consecutive points withing quiescent curve
            while any(curve.iloc[ix_range[0]]['Flux']-err >= q_curve['Quiescent Flux']) & ix_range[0] > 0 or (consec < 1 & ix_range[0] >0):
                err = curve.iloc[ix_range[0]]['Flux Error']
                
                #if flux < q add 1 to consecutive points
                if any(curve.iloc[ix_range[0]]['Flux']- err < q_curve['Quiescent Flux']) :
                    consec +=1
                else: 
                    consec = 0
                if (curve.iloc[ix_range[0]+1]['Flux']-curve.iloc[ix_range[0]]['Flux']) > 1000: #don't understand this statement much, how do I get it to break correctly?
                    break               
                ix_range = [ix_range[0] - 1] + ix_range
                
                # go forwards
            consec = 0 
            err = curve.iloc[ix_range[-1]]['Flux Error']
            while any(curve.iloc[ix_range[-1]]['Flux']-err >= q_curve['Quiescent Flux']) & ix_range[-1] != len(curve)-1 or (consec <1 & ix_range[-1]!= len(curve)-1):
                err = curve.iloc[ix_range[-1]]['Flux Error']
                if any(curve.iloc[ix_range[-1]]['Flux']-err < q_curve['Quiescent Flux']):
                    consec += 1
                else: 
                    consec = 0
                if curve.iloc[ix_range[-1]+1]['Flux']-curve.iloc[ix_range[-1]]['Flux'] > 1000: #don't understand this statement much, how do I get it to break correctly?
                    break
                ix_range = ix_range + [ix_range[-1] + 1]
                
            flux_ix += ix_range
        ix = np.unique(flux_ix)
        flare_ranges += find_ix_ranges(list(np.array(ix).flatten()))
    return(flare_ranges,flareFlux)

                

In [ ]:
def energyCalculation(flare_ranges,flux_values,binsize=20):
    """This function will integrate the flux over time to produce the energy for each flare.
        Stage: debugging"""
    
    energies = []
    for flare in flare_ranges: 
        int_flux = (binsize*flux_values).sum()
        energies.append(int_flux)
        
        #NEXT: don't forget to calculate errors
        
    return(energies)

In [78]:
def FlareTable(cleaned_data,flare_ranges,flareFlux,energies):
    """This function will build a table of all flares.
        Stage: debugging"""
    
    #Generate quiescent and flaring curves in order to index time values
    curves = generate_curves(cleaned_data)
    flaring = curves[1]
    
    #Create empty lists to use as columns for table

    tstart = []
    tstop = []
    duration = []
    energy = []
    
    #Iterate through each individual flare in flare ranges to get values, append to corresponding list
    for flare in flare_ranges: 
        ind_tstart = flare[0]
        ind_tstop = flare[-1]
        ind_duration = flare[-1]-flare[0]
        ind_energy = energies[flare]
        tstart.append(ind_tstart)
        duration.append(ind_duration)
        energy.append(ind_energy)
    #Label flares    
    flareID = [range(len(flare_ranges))] 
    
    #Build flare table
    flareTable = pd.DataFrame({
                                "ID": flareID,
                                "Start Time": tstart,
                                "Stop Time": tstop,
                                "Duration": duration,
                                "Total Energy": energy
    })
    
    return(flareTable)
        

In [1]:
def make_FlarePlots(flareTable, cleaned_data,flare_ix):
    """This function will create plots of all detected flares.
        Parameter flare_ix will be indices of flares that can be 
        used to plot from cleaned data.
        Stage: very rough working draft"""
    
    for row in flareTable: 
        #Set ranges to give 3 mins of data on either side of flare
        time_range = (flareTable["Start Time"] - 3 mins (-12 index),flareTable["Stop Time"] + 3 mins (+12index))
        flux_range = (flare_ix - 12index, flare_ix + 12index)
        
        plt.xlabel("Time (UTC)")
        plt.ylabel("Flux (e/s)")
        plt.figure(figsize=(14,3))
        plt.text(location, "Time:" flareTable["Start Time"])
        plt.text(location, "Duration:"flareTable["Duration"])
        plt.text(location, "Energy:"flareTable["Total Energy"])
        plt.plot(time_range, flux_range)
        
        
        
        

In [77]:
TESSDATA = '/Users/katborski/Documents/GitHub/AFPSC/TESS/tess2021232031932-s0042-0000000250081915-0213-a_fast-lc.fits'
cleaned_data = dataClean(TESSDATA)

dataset = generate_Qcurve(cleaned_data)
q_curve = dataset[0]
curve = dataset[1]

flareFinder(curve,q_curve,3)

IndexError: single positional indexer is out-of-bounds